<a href="https://colab.research.google.com/github/SYEDA-NIGHAT/Assignment_5_Projects/blob/main/Assignment_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 5

In [1]:
!pip install streamlit pyngrok cryptography

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 51.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 51.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 4.2 MB/s eta 0:00:00


In [4]:
!pip install pyngrok
from pyngrok import ngrok

ngrok.set_auth_token("2xSoBBNs493lMjnvj1ELuyHMby2_59nsqz4uq5iZXMnHfnMUm")


In [6]:
code = '''
import streamlit as st
import hashlib
from cryptography.fernet import Fernet

if 'KEY' not in st.session_state:
    st.session_state.KEY = Fernet.generate_key()
    st.session_state.cipher = Fernet(st.session_state.KEY)

if 'stored_data' not in st.session_state:
    st.session_state.stored_data = {}

if 'failed_attempts' not in st.session_state:
    st.session_state.failed_attempts = 0

def hash_passkey(passkey):
    return hashlib.sha256(passkey.encode()).hexdigest()

def encrypt_data(text):
    return st.session_state.cipher.encrypt(text.encode()).decode()

def decrypt_data(encrypted_text, passkey):
    hashed_pass = hash_passkey(passkey)
    for data in st.session_state.stored_data.values():
        if data["encrypted_text"] == encrypted_text and data["passkey"] == hashed_pass:
            st.session_state.failed_attempts = 0
            return st.session_state.cipher.decrypt(encrypted_text.encode()).decode()
    st.session_state.failed_attempts += 1
    return None

st.title("🔒 Secure Data Encryption System")

menu = ["Home", "Store Data", "Retrieve Data", "Login"]
choice = st.sidebar.selectbox("Navigation", menu)

if choice == "Home":
    st.subheader("🏠 Welcome")
    st.write("Securely store and retrieve data using passkeys.")

elif choice == "Store Data":
    st.subheader("📂 Store Data")
    data = st.text_area("Enter data:")
    passkey = st.text_input("Enter passkey:", type="password")

    if st.button("Encrypt & Store"):
        if data and passkey:
            encrypted = encrypt_data(data)
            hashed = hash_passkey(passkey)
            st.session_state.stored_data[encrypted] = {"encrypted_text": encrypted, "passkey": hashed}
            st.success("✅ Data stored securely!")
            st.code(encrypted, language='text')
        else:
            st.error("Both fields required!")

elif choice == "Retrieve Data":
    st.subheader("🔍 Retrieve Data")
    encrypted = st.text_area("Enter encrypted text:")
    passkey = st.text_input("Enter your passkey:", type="password")

    if st.button("Decrypt"):
        if encrypted and passkey:
            result = decrypt_data(encrypted, passkey)
            if result:
                st.success(f"✅ Decrypted Data:\\n{result}")
            else:
                st.error(f"❌ Incorrect! Attempts left: {3 - st.session_state.failed_attempts}")
                if st.session_state.failed_attempts >= 3:
                    st.warning("🔒 Too many failed attempts! Redirecting to Login.")
                    st.experimental_rerun()
        else:
            st.error("Both fields required!")

elif choice == "Login":
    st.subheader("🔑 Login")
    master_pass = st.text_input("Enter master password:", type="password")
    if st.button("Login"):
        if master_pass == "admin123":
            st.session_state.failed_attempts = 0
            st.success("✅ Reauthorized.")
            st.experimental_rerun()
        else:
            st.error("❌ Wrong master password.")
'''

# Save to app.py
with open("app.py", "w") as f:
    f.write(code)


In [8]:
!streamlit run app.py &>/dev/null &
public_url = ngrok.connect(8501)
print("🔗 Open your app here:", public_url)

🔗 Open your app here: NgrokTunnel: "https://d195-35-230-59-210.ngrok-free.app" -> "http://localhost:8501"
